---
## BLOCK 1: ENVIRONMENT SETUP & IMPORTS
**Function:** `setup_environment()`

**Purpose:** Import all required libraries and set configuration parameters

**Inputs:** None

**Outputs:** Imported modules and libraries

**Dependencies:** None (first step)

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib
import time
from datetime import datetime

# Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Model selection and validation
from sklearn.model_selection import train_test_split, cross_val_score

# Classification models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import xgboost as xgb

# Metrics
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve,
    log_loss,
    cohen_kappa_score,
    matthews_corrcoef
)

# Resampling
from imblearn.combine import SMOTEENN

# Statistical tests
from scipy.stats import anderson, zscore

# Hyperparameter optimization
import optuna

# Configuration
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Environment setup complete")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## BLOCK 2: DATA LOADING & VALIDATION
**Function:** `load_data(train_path, test_path)`

**Purpose:** Load training and test datasets from CSV files with basic validation

**Inputs:**
- `train_path` (str): Path to training dataset
- `test_path` (str): Path to test dataset

**Outputs:**
- `train_df` (pd.DataFrame): Training dataset
- `test_df` (pd.DataFrame): Test dataset

**Dependencies:** None

In [ ]:
# Load datasets
train_path = 'churn-bigml-80.csv'
test_path = 'churn-bigml-20.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print("=" * 60)
print("DATA LOADING SUMMARY")
print("=" * 60)
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")
print(f"\nColumns: {list(train_df.columns)}")
print("\n✓ Data loaded successfully")

---
## BLOCK 3: DATA QUALITY VALIDATION
**Function:** `validate_data_quality(df, dataset_name)`

**Purpose:** Perform comprehensive data quality checks

**Inputs:**
- `df` (pd.DataFrame): Dataset to validate
- `dataset_name` (str): Name for logging

**Outputs:**
- `quality_report` (dict): Dictionary with quality metrics

**Dependencies:** BLOCK 2 (requires loaded data)

In [ ]:
def validate_data_quality(df, dataset_name="Dataset"):
    """Validate data quality and return comprehensive report"""
    report = {
        'dataset_name': dataset_name,
        'shape': df.shape,
        'missing_values': df.isna().sum().to_dict(),
        'duplicate_count': df.duplicated().sum(),
        'data_types': df.dtypes.to_dict(),
        'unique_counts': df.nunique().to_dict()
    }
    
    print("=" * 60)
    print(f"{dataset_name.upper()} - DATA QUALITY REPORT")
    print("=" * 60)
    print(f"Shape: {report['shape']}")
    print(f"Missing values: {sum(report['missing_values'].values())} total")
    print(f"Duplicate rows: {report['duplicate_count']}")
    print(f"\nData types:\n{df.dtypes.value_counts()}")
    print("\n✓ Data quality validation complete")
    
    return report

# Validate both datasets
train_report = validate_data_quality(train_df, "Training Set")
test_report = validate_data_quality(test_df, "Test Set")

---
## BLOCK 4: EXPLORATORY DATA ANALYSIS (EDA)
**Function:** `perform_eda(df, target_col, save_plots)`

**Purpose:** Generate comprehensive EDA visualizations and statistics

**Inputs:**
- `df` (pd.DataFrame): Dataset to analyze
- `target_col` (str): Target column name
- `save_plots` (bool): Whether to save plots

**Outputs:**
- `eda_summary` (dict): Statistical summary
- Visualization files (optional)

**Dependencies:** BLOCK 2 (requires loaded data)

In [ ]:
# Display basic statistics
print("=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
print(train_df.describe())

# Identify categorical and numerical columns
categorical_cols = ["State", "Area code", "International plan", "Voice mail plan", 
                    "Customer service calls", "Churn"]
numerical_cols = [col for col in train_df.columns if col not in categorical_cols]

print(f"\nCategorical columns ({len(categorical_cols)}): {categorical_cols}")
print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")

In [ ]:
# Visualize categorical features
plt.figure(figsize=(15, 40))
plot_num = 1
for col in categorical_cols:
    plt.subplot(10, 2, plot_num)
    sns.countplot(data=train_df, x=col)
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)
    plot_num += 1
plt.tight_layout()
plt.show()

print("✓ Categorical features visualized")

In [ ]:
# Visualize numerical features
plt.figure(figsize=(15, 40))
plot_num = 1
for col in numerical_cols:
    plt.subplot(10, 2, plot_num)
    sns.histplot(data=train_df, x=col, bins=25)
    plt.title(f'Distribution of {col}')
    plot_num += 1
plt.tight_layout()
plt.show()

print("✓ Numerical features visualized")

In [ ]:
# Visualize churn distribution by categorical features
plt.figure(figsize=(15, 8))
sns.countplot(x='State', hue='Churn', data=train_df, palette='Set2')
plt.title('Churn Distribution by State', fontsize=16)
plt.xlabel('State', fontsize=14)
plt.ylabel('Count of Customers', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(15, 8))
sns.countplot(x='Customer service calls', hue='Churn', data=train_df, palette='Set2')
plt.title('Churn Distribution by Customer Service Calls', fontsize=16)
plt.xlabel('Number of Service Calls', fontsize=14)
plt.ylabel('Count of Customers', fontsize=14)
plt.tight_layout()
plt.show()

print("✓ Churn distribution analysis complete")

---
## BLOCK 5: CATEGORICAL ENCODING
**Function:** `encode_categorical_features(train_df, test_df, binary_cols, onehot_cols)`

**Purpose:** Encode categorical variables using appropriate strategies

**Inputs:**
- `train_df` (pd.DataFrame): Training dataset
- `test_df` (pd.DataFrame): Test dataset
- `binary_cols` (list): Columns for binary encoding
- `onehot_cols` (list): Columns for one-hot encoding

**Outputs:**
- `train_encoded` (pd.DataFrame): Encoded training data
- `test_encoded` (pd.DataFrame): Encoded test data
- `encoders` (dict): Fitted encoder objects

**Dependencies:** BLOCK 2 (requires loaded data)

In [ ]:
print("=" * 60)
print("CATEGORICAL ENCODING")
print("=" * 60)

# Create copies to avoid modifying originals
train_encoded = train_df.copy()
test_encoded = test_df.copy()

# Step 1: Convert target to binary integer
train_encoded['Churn'] = train_encoded['Churn'].astype(int)
test_encoded['Churn'] = test_encoded['Churn'].astype(int)
print("✓ Target variable encoded (False=0, True=1)")

# Step 2: Binary encoding for Yes/No columns
binary_mapping = {'No': 0, 'Yes': 1}
train_encoded['International plan'] = train_encoded['International plan'].map(binary_mapping)
test_encoded['International plan'] = test_encoded['International plan'].map(binary_mapping)

train_encoded['Voice mail plan'] = train_encoded['Voice mail plan'].map(binary_mapping)
test_encoded['Voice mail plan'] = test_encoded['Voice mail plan'].map(binary_mapping)
print("✓ Binary features encoded (No=0, Yes=1)")

# Step 3: One-Hot Encoding for State and Area Code
encoders = {}

# State encoder
encoder_state = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_states_train = encoder_state.fit_transform(train_encoded[['State']])
encoded_states_test = encoder_state.transform(test_encoded[['State']])

state_columns = encoder_state.get_feature_names_out(['State'])
train_states_df = pd.DataFrame(encoded_states_train, columns=state_columns, index=train_encoded.index)
test_states_df = pd.DataFrame(encoded_states_test, columns=state_columns, index=test_encoded.index)

# Area code encoder
encoder_area = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_area_train = encoder_area.fit_transform(train_encoded[['Area code']])
encoded_area_test = encoder_area.transform(test_encoded[['Area code']])

area_columns = encoder_area.get_feature_names_out(['Area code'])
train_area_df = pd.DataFrame(encoded_area_train, columns=area_columns, index=train_encoded.index)
test_area_df = pd.DataFrame(encoded_area_test, columns=area_columns, index=test_encoded.index)

# Remove original columns and concatenate encoded ones
train_encoded = train_encoded.drop(['State', 'Area code'], axis=1)
test_encoded = test_encoded.drop(['State', 'Area code'], axis=1)

train_encoded = pd.concat([train_encoded, train_states_df, train_area_df], axis=1)
test_encoded = pd.concat([test_encoded, test_states_df, test_area_df], axis=1)

# Store encoders
encoders['state_encoder'] = encoder_state
encoders['area_encoder'] = encoder_area

print(f"✓ One-hot encoding complete")
print(f"  - State: {len(state_columns)} features")
print(f"  - Area code: {len(area_columns)} features")
print(f"\nFinal shapes:")
print(f"  Training: {train_encoded.shape}")
print(f"  Test: {test_encoded.shape}")

---
## BLOCK 6: OUTLIER DETECTION & REMOVAL
**Function:** `remove_outliers(df, method, columns)`

**Purpose:** Detect and remove outliers using statistical methods

**Inputs:**
- `df` (pd.DataFrame): Dataset to clean
- `method` (str): 'zscore' or 'iqr'
- `columns` (list): Columns to check for outliers

**Outputs:**
- `df_cleaned` (pd.DataFrame): Dataset without outliers
- `outlier_report` (dict): Summary of removed data

**Dependencies:** BLOCK 5 (requires encoded data)

In [ ]:
print("=" * 60)
print("OUTLIER DETECTION & REMOVAL")
print("=" * 60)

# Define columns to check for outliers
outlier_columns = [
    'Account length', 'Total day minutes', 'Total day calls', 'Total day charge',
    'Total eve minutes', 'Total eve calls', 'Total eve charge',
    'Total night minutes', 'Total night calls', 'Total night charge',
    'Total intl minutes', 'Total intl calls', 'Total intl charge'
]

# Visualize outliers before removal
plt.figure(figsize=(20, 15))
for i, column in enumerate(outlier_columns, 1):
    plt.subplot(4, 4, i)
    sns.boxplot(data=train_encoded, y=column, color='skyblue')
    plt.title(f'{column}')
plt.tight_layout()
plt.suptitle('Outliers - Before Removal', fontsize=16, y=1.001)
plt.show()

# Test for normality using Anderson-Darling test
normal_columns = []
non_normal_columns = []

for column in outlier_columns:
    result = anderson(train_encoded[column])
    if result.statistic < result.critical_values[2]:  # 5% significance level
        normal_columns.append(column)
        print(f"✓ {column}: Normal distribution (AD stat={result.statistic:.3f})")
    else:
        non_normal_columns.append(column)
        print(f"✗ {column}: Non-normal distribution (AD stat={result.statistic:.3f})")

print(f"\nNormal columns: {len(normal_columns)}")
print(f"Non-normal columns: {len(non_normal_columns)}")

In [ ]:
# Remove outliers from normal columns using Z-score method
original_shape = train_encoded.shape[0]

if normal_columns:
    z_scores = zscore(train_encoded[normal_columns])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    train_encoded = train_encoded[filtered_entries]
    print(f"✓ Z-score outlier removal: {original_shape - train_encoded.shape[0]} rows removed")

# Remove outliers from non-normal columns using IQR method
def remove_outliers_iqr(df, columns, multiplier=1.5):
    """Remove outliers using IQR method"""
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

if non_normal_columns:
    shape_before = train_encoded.shape[0]
    train_encoded = remove_outliers_iqr(train_encoded, non_normal_columns)
    print(f"✓ IQR outlier removal: {shape_before - train_encoded.shape[0]} rows removed")

print(f"\nFinal training shape: {train_encoded.shape}")
print(f"Total rows removed: {original_shape - train_encoded.shape[0]} ({((original_shape - train_encoded.shape[0]) / original_shape * 100):.2f}%)")

In [ ]:
# Visualize outliers after removal
plt.figure(figsize=(20, 15))
for i, column in enumerate(outlier_columns, 1):
    plt.subplot(4, 4, i)
    sns.boxplot(data=train_encoded, y=column, color='lightgreen')
    plt.title(f'{column}')
plt.tight_layout()
plt.suptitle('Outliers - After Removal', fontsize=16, y=1.001)
plt.show()

print("✓ Outlier removal complete")

---
## BLOCK 7: FEATURE ENGINEERING
**Function:** `engineer_features(df)`

**Purpose:** Create derived features based on domain knowledge

**Inputs:**
- `df` (pd.DataFrame): Dataset to enhance

**Outputs:**
- `df_engineered` (pd.DataFrame): Dataset with new features

**Dependencies:** BLOCK 6 (requires cleaned data)

In [ ]:
print("=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)

# Create new features
# 1. Total calls across all periods
train_encoded['Total calls'] = (train_encoded['Total day calls'] + 
                                  train_encoded['Total eve calls'] + 
                                  train_encoded['Total night calls'] + 
                                  train_encoded['Total intl calls'])

test_encoded['Total calls'] = (test_encoded['Total day calls'] + 
                                test_encoded['Total eve calls'] + 
                                test_encoded['Total night calls'] + 
                                test_encoded['Total intl calls'])

print("✓ Created 'Total calls' feature")

# 2. Total charge across all periods
train_encoded['Total charge'] = (train_encoded['Total day charge'] + 
                                   train_encoded['Total eve charge'] + 
                                   train_encoded['Total night charge'] + 
                                   train_encoded['Total intl charge'])

test_encoded['Total charge'] = (test_encoded['Total day charge'] + 
                                 test_encoded['Total eve charge'] + 
                                 test_encoded['Total night charge'] + 
                                 test_encoded['Total intl charge'])

print("✓ Created 'Total charge' feature")

# 3. Customer service calls rate (normalized by account length)
train_encoded['CScalls Rate'] = train_encoded['Customer service calls'] / train_encoded['Account length']
test_encoded['CScalls Rate'] = test_encoded['Customer service calls'] / test_encoded['Account length']

print("✓ Created 'CScalls Rate' feature")

print(f"\nNew features summary:")
print(f"  - Total calls: Aggregates all call types")
print(f"  - Total charge: Aggregates all charges (validates hypothesis 2)")
print(f"  - CScalls Rate: Service calls per account day (validates hypothesis 1)")

In [ ]:
# Visualize engineered features
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

sns.histplot(data=train_encoded, x='Total charge', hue='Churn', kde=True, bins=30, ax=axes[0])
axes[0].set_title('Total Charge Distribution by Churn')

sns.histplot(data=train_encoded, x='Total calls', hue='Churn', kde=True, bins=30, ax=axes[1])
axes[1].set_title('Total Calls Distribution by Churn')

sns.histplot(data=train_encoded, x='CScalls Rate', hue='Churn', kde=True, bins=30, ax=axes[2])
axes[2].set_title('CS Calls Rate Distribution by Churn')

plt.tight_layout()
plt.show()

print("✓ Feature engineering complete")

---
## BLOCK 8: FEATURE SELECTION (CORRELATION-BASED)
**Function:** `select_features_correlation(df, threshold, features_to_drop)`

**Purpose:** Remove highly correlated and redundant features

**Inputs:**
- `df` (pd.DataFrame): Dataset with all features
- `threshold` (float): Correlation threshold
- `features_to_drop` (list): Manual feature list to remove

**Outputs:**
- `df_selected` (pd.DataFrame): Dataset with selected features
- `dropped_features` (list): List of removed features

**Dependencies:** BLOCK 7 (requires engineered features)

In [ ]:
print("=" * 60)
print("CORRELATION-BASED FEATURE SELECTION")
print("=" * 60)

# Analyze correlations before removal
correlation_cols = [
    'Account length', 'Total day minutes', 'Voice mail plan', 'Number vmail messages',
    'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls',
    'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge',
    'Total intl minutes', 'Total intl calls', 'Total intl charge'
]

corr_matrix = train_encoded[correlation_cols].corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", center=0)
plt.title('Correlation Matrix - Before Feature Removal')
plt.tight_layout()
plt.show()

# Remove highly correlated features (correlation > 0.9 with engineered features)
correlated_features = [
    'Total day minutes',    # Highly correlated with Total day charge (>0.99)
    'Total eve minutes',    # Highly correlated with Total eve charge (>0.99)
    'Total night minutes',  # Highly correlated with Total night charge (>0.99)
    'Total intl minutes',   # Highly correlated with Total intl charge (>0.99)
    'Voice mail plan'       # Redundant with Number vmail messages
]

train_encoded.drop(correlated_features, axis=1, inplace=True)
test_encoded.drop(correlated_features, axis=1, inplace=True)

print(f"✓ Removed {len(correlated_features)} highly correlated features:")
for feat in correlated_features:
    print(f"  - {feat}")

# Visualize correlations after removal
remaining_numeric_cols = [
    'Account length', 'Number vmail messages', 'Total day calls', 'Total day charge',
    'Total eve calls', 'Total eve charge', 'Total night calls', 'Total night charge',
    'Total intl calls', 'Total intl charge', 'Total calls', 'Total charge', 'CScalls Rate'
]

corr_matrix_clean = train_encoded[remaining_numeric_cols].corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix_clean, annot=True, cmap='coolwarm', fmt=".2f", center=0)
plt.title('Correlation Matrix - After Feature Removal')
plt.tight_layout()
plt.show()

print(f"\nFinal shapes:")
print(f"  Training: {train_encoded.shape}")
print(f"  Test: {test_encoded.shape}")

---
## BLOCK 9: DATA PREPARATION FOR MODELING
**Function:** `prepare_data_for_modeling(train_df, test_df, target_col)`

**Purpose:** Split features/target and prepare final datasets for modeling

**Inputs:**
- `train_df` (pd.DataFrame): Processed training dataset
- `test_df` (pd.DataFrame): Processed test dataset
- `target_col` (str): Target column name

**Outputs:**
- `X_train` (pd.DataFrame): Training features
- `y_train` (pd.Series): Training labels
- `X_test` (pd.DataFrame): Test features
- `y_test` (pd.Series): Test labels

**Dependencies:** BLOCK 8 (requires feature-selected data)

In [ ]:
print("=" * 60)
print("DATA PREPARATION FOR MODELING")
print("=" * 60)

# Separate features and target
X_train = train_encoded.drop(['Churn'], axis=1)
y_train = train_encoded['Churn']

X_test = test_encoded.drop(['Churn'], axis=1)
y_test = test_encoded['Churn']

print(f"Training set:")
print(f"  Features: {X_train.shape}")
print(f"  Target: {y_train.shape}")
print(f"  Class distribution: {y_train.value_counts().to_dict()}")

print(f"\nTest set:")
print(f"  Features: {X_test.shape}")
print(f"  Target: {y_test.shape}")
print(f"  Class distribution: {y_test.value_counts().to_dict()}")

# Check class balance
churn_rate_train = y_train.mean() * 100
churn_rate_test = y_test.mean() * 100

print(f"\nChurn rate:")
print(f"  Training: {churn_rate_train:.2f}%")
print(f"  Test: {churn_rate_test:.2f}%")

if churn_rate_train < 30:
    print("\n⚠ Warning: Imbalanced dataset detected. Resampling recommended.")

---
## BLOCK 10: DATA BALANCING
**Function:** `balance_dataset(X, y, sampling_strategy, random_state)`

**Purpose:** Balance imbalanced dataset using SMOTEENN

**Inputs:**
- `X` (pd.DataFrame): Features
- `y` (pd.Series): Labels
- `sampling_strategy` (float): Desired minority/majority ratio
- `random_state` (int): Random seed

**Outputs:**
- `X_resampled` (np.ndarray): Balanced features
- `y_resampled` (np.ndarray): Balanced labels
- `balance_report` (dict): Class distribution before/after

**Dependencies:** BLOCK 9 (requires separated features and target)

In [ ]:
print("=" * 60)
print("DATA BALANCING (SMOTEENN)")
print("=" * 60)

# Store original distribution
original_distribution = y_train.value_counts().to_dict()
print(f"Original class distribution: {original_distribution}")

# Apply SMOTEENN (SMOTE + Edited Nearest Neighbors)
desired_ratio = 30 / 70  # Target 30% minority class
smote_enn = SMOTEENN(sampling_strategy=desired_ratio, random_state=RANDOM_STATE)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

# Report new distribution
new_distribution = pd.Series(y_resampled).value_counts().to_dict()
print(f"\nResampled class distribution: {new_distribution}")
print(f"New dataset size: {len(y_resampled)} samples")
print(f"Minority class ratio: {(new_distribution.get(1, 0) / len(y_resampled) * 100):.2f}%")

print("\n✓ Data balancing complete")

---
## BLOCK 11: FEATURE SCALING
**Function:** `scale_features(X_train, X_test, return_scaler)`

**Purpose:** Standardize features using StandardScaler

**Inputs:**
- `X_train` (np.ndarray): Training features
- `X_test` (np.ndarray): Test features
- `return_scaler` (bool): Whether to return scaler object

**Outputs:**
- `X_train_scaled` (np.ndarray): Scaled training features
- `X_test_scaled` (np.ndarray): Scaled test features
- `scaler` (StandardScaler): Fitted scaler (optional)

**Dependencies:** BLOCK 10 (requires balanced data)

In [ ]:
print("=" * 60)
print("FEATURE SCALING")
print("=" * 60)

# Initialize and fit scaler on training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print(f"✓ Features scaled using StandardScaler")
print(f"  Training shape: {X_train_scaled.shape}")
print(f"  Test shape: {X_test_scaled.shape}")
print(f"\nScaling parameters:")
print(f"  Mean: {scaler.mean_[:5]}... (first 5 features)")
print(f"  Std: {scaler.scale_[:5]}... (first 5 features)")

# Verify scaling
print(f"\nScaled data statistics:")
print(f"  Mean: {X_train_scaled.mean():.6f}")
print(f"  Std: {X_train_scaled.std():.6f}")

---
## BLOCK 12: BASELINE MODEL COMPARISON
**Function:** `compare_baseline_models(X_train, y_train, X_test, y_test)`

**Purpose:** Train and compare multiple classification algorithms

**Inputs:**
- `X_train` (np.ndarray): Scaled training features
- `y_train` (np.ndarray): Training labels
- `X_test` (np.ndarray): Scaled test features
- `y_test` (np.ndarray): Test labels

**Outputs:**
- `results_df` (pd.DataFrame): Comparison metrics for all models
- `trained_models` (dict): Dictionary of fitted models

**Dependencies:** BLOCK 11 (requires scaled data)

In [ ]:
print("=" * 60)
print("BASELINE MODEL COMPARISON")
print("=" * 60)

# Define models to compare
models = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE),
    "Support Vector Machine": SVC(class_weight='balanced', random_state=RANDOM_STATE),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced', random_state=RANDOM_STATE),
    "Random Forest": RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE),
    "Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "AdaBoost": AdaBoostClassifier(random_state=RANDOM_STATE),
    "XGBoost": XGBClassifier(random_state=RANDOM_STATE),
    "Neural Network": MLPClassifier(random_state=RANDOM_STATE, max_iter=1000)
}

# Train and evaluate each model
results = []
trained_models = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    
    # Train
    start_time = time.time()
    model.fit(X_train_scaled, y_resampled)
    train_time = time.time() - start_time
    
    # Predict
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Train Time (s)': train_time
    })
    
    trained_models[model_name] = model
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(confusion_matrix(y_test, y_pred))

# Create results DataFrame
results_df = pd.DataFrame(results).sort_values('Accuracy', ascending=False)

print("\n" + "=" * 60)
print("MODEL COMPARISON RESULTS")
print("=" * 60)
print(results_df.to_string(index=False))
print(f"\n✓ Best model: {results_df.iloc[0]['Model']} ({results_df.iloc[0]['Accuracy']:.4f})")

---
## BLOCK 13: HYPERPARAMETER OPTIMIZATION
**Function:** `optimize_hyperparameters(model_type, X_train, y_train, X_test, y_test, n_trials)`

**Purpose:** Optimize model hyperparameters using Optuna

**Inputs:**
- `model_type` (str): Type of model ('random_forest' or 'xgboost')
- `X_train`, `y_train`: Training data
- `X_test`, `y_test`: Test data
- `n_trials` (int): Number of optimization trials

**Outputs:**
- `best_model` (estimator): Model with optimized hyperparameters
- `best_params` (dict): Best parameters found
- `study` (optuna.Study): Optimization study object

**Dependencies:** BLOCK 12 (requires baseline comparison)

In [ ]:
print("=" * 60)
print("HYPERPARAMETER OPTIMIZATION - RANDOM FOREST")
print("=" * 60)

def objective_rf(trial):
    """Optuna objective function for Random Forest"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'max_depth': trial.suggest_int('max_depth', 5, 30, step=5),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'random_state': RANDOM_STATE
    }
    
    model = RandomForestClassifier(**params)
    model.fit(X_train_scaled, y_resampled)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Run optimization
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50, show_progress_bar=True)

print(f"\n✓ Optimization complete")
print(f"Best accuracy: {study_rf.best_value:.4f}")
print(f"Best parameters: {study_rf.best_params}")

# Train final model with best parameters
best_rf_model = RandomForestClassifier(**study_rf.best_params, random_state=RANDOM_STATE)
best_rf_model.fit(X_train_scaled, y_resampled)

# Evaluate
y_pred_rf = best_rf_model.predict(X_test_scaled)
print(f"\n Random Forest - Best Model Performance:")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

In [ ]:
print("=" * 60)
print("HYPERPARAMETER OPTIMIZATION - XGBOOST")
print("=" * 60)

def objective_xgb(trial):
    """Optuna objective function for XGBoost"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': RANDOM_STATE
    }
    
    model = XGBClassifier(**params)
    model.fit(X_train_scaled, y_resampled)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Run optimization
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)

print(f"\n✓ Optimization complete")
print(f"Best accuracy: {study_xgb.best_value:.4f}")
print(f"Best parameters: {study_xgb.best_params}")

# Train final model with best parameters
best_xgb_model = XGBClassifier(**study_xgb.best_params, random_state=RANDOM_STATE)
best_xgb_model.fit(X_train_scaled, y_resampled)

# Evaluate
y_pred_xgb = best_xgb_model.predict(X_test_scaled)
print(f"\nXGBoost - Best Model Performance:")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

---
## BLOCK 14: FEATURE IMPORTANCE ANALYSIS
**Function:** `extract_feature_importance(model, feature_names, top_n)`

**Purpose:** Extract and visualize feature importances

**Inputs:**
- `model` (estimator): Trained model with feature_importances_
- `feature_names` (list): Feature names
- `top_n` (int): Number of top features to return

**Outputs:**
- `importance_df` (pd.DataFrame): Feature importance scores
- `top_features` (list): Top N feature names

**Dependencies:** BLOCK 13 (requires trained models)

In [ ]:
print("=" * 60)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 60)

# Get feature names
feature_names = X_train.columns.tolist()

# Random Forest Feature Importance
rf_importances = best_rf_model.feature_importances_
rf_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_importances
}).sort_values('Importance', ascending=False)

print("\nRandom Forest - Top 15 Important Features:")
print(rf_importance_df.head(15).to_string(index=False))

# Visualize RF importance
plt.figure(figsize=(12, 8))
top_n = 20
sorted_idx = rf_importances.argsort()[::-1][:top_n]
plt.barh(range(top_n), rf_importances[sorted_idx])
plt.yticks(range(top_n), [feature_names[i] for i in sorted_idx])
plt.xlabel('Importance')
plt.title('Random Forest - Top 20 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# XGBoost Feature Importance
xgb_importances = best_xgb_model.feature_importances_
xgb_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': xgb_importances
}).sort_values('Importance', ascending=False)

print("\nXGBoost - Top 15 Important Features:")
print(xgb_importance_df.head(15).to_string(index=False))

# Visualize XGBoost importance
plt.figure(figsize=(12, 8))
sorted_idx = xgb_importances.argsort()[::-1][:top_n]
plt.barh(range(top_n), xgb_importances[sorted_idx])
plt.yticks(range(top_n), [feature_names[i] for i in sorted_idx])
plt.xlabel('Importance')
plt.title('XGBoost - Top 20 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Extract top features for final model
top_features_rf = rf_importance_df.head(10)['Feature'].tolist()
top_features_xgb = xgb_importance_df.head(14)['Feature'].tolist()

print(f"\n✓ Feature importance analysis complete")

---
## BLOCK 15: FINAL MODEL EVALUATION
**Function:** `evaluate_model_comprehensive(model, X_test, y_test, model_name)`

**Purpose:** Comprehensive model evaluation with multiple metrics

**Inputs:**
- `model` (estimator): Trained model
- `X_test` (np.ndarray): Test features
- `y_test` (np.ndarray): True labels
- `model_name` (str): Model name for reporting

**Outputs:**
- `metrics_dict` (dict): All evaluation metrics
- Displays plots and reports

**Dependencies:** BLOCK 13 (requires trained models)

In [ ]:
print("=" * 60)
print("COMPREHENSIVE MODEL EVALUATION - XGBOOST")
print("=" * 60)

# Select top features
X_test_selected = X_test[top_features_xgb]
X_resampled_selected = X_resampled[top_features_xgb]

# Scale
X_train_scaled_final = scaler.fit_transform(X_resampled_selected)
X_test_scaled_final = scaler.transform(X_test_selected)

# Train final model
best_xgb_model.fit(X_train_scaled_final, y_resampled)
y_pred = best_xgb_model.predict(X_test_scaled_final)
y_pred_proba = best_xgb_model.predict_proba(X_test_scaled_final)[:, 1]

# Calculate comprehensive metrics
metrics = {}
metrics['Accuracy'] = accuracy_score(y_test, y_pred)
metrics['ROC AUC'] = roc_auc_score(y_test, y_pred_proba)
metrics['Log Loss'] = log_loss(y_test, y_pred_proba)
metrics['Cohen Kappa'] = cohen_kappa_score(y_test, y_pred)
metrics['Matthews Corr Coef'] = matthews_corrcoef(y_test, y_pred)

print("Performance Metrics:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {metrics["ROC AUC"]:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Comprehensive evaluation complete")

---
## BLOCK 16: CROSS-VALIDATION
**Function:** `cross_validate_model(model, X, y, cv, scoring)`

**Purpose:** Perform k-fold cross-validation

**Inputs:**
- `model` (estimator): Model to validate
- `X` (np.ndarray): Features
- `y` (np.ndarray): Labels
- `cv` (int): Number of folds
- `scoring` (str): Scoring metric

**Outputs:**
- `cv_results` (dict): CV scores and statistics

**Dependencies:** BLOCK 13 (requires trained models)

In [ ]:
print("=" * 60)
print("CROSS-VALIDATION ANALYSIS")
print("=" * 60)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(
    best_xgb_model,
    X_train_scaled_final,
    y_resampled,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

print("Cross-Validation Results:")
print(f"  Individual fold scores: {cv_scores}")
print(f"  Mean ROC AUC: {cv_scores.mean():.4f}")
print(f"  Std ROC AUC: {cv_scores.std():.4f}")
print(f"  95% Confidence Interval: [{cv_scores.mean() - 1.96*cv_scores.std():.4f}, {cv_scores.mean() + 1.96*cv_scores.std():.4f}]")

# Visualize CV scores
plt.figure(figsize=(10, 6))
plt.bar(range(1, 6), cv_scores, alpha=0.7, color='skyblue', edgecolor='black')
plt.axhline(y=cv_scores.mean(), color='red', linestyle='--', label=f'Mean: {cv_scores.mean():.4f}')
plt.xlabel('Fold')
plt.ylabel('ROC AUC Score')
plt.title('5-Fold Cross-Validation Scores')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Cross-validation complete")

---
## BLOCK 17: MODEL PERSISTENCE (SAVE)
**Function:** `save_model_artifacts(model, scaler, encoders, feature_names, output_dir)`

**Purpose:** Save trained model and all preprocessing artifacts

**Inputs:**
- `model` (estimator): Trained model
- `scaler` (StandardScaler): Fitted scaler
- `encoders` (dict): Fitted encoders
- `feature_names` (list): Selected feature names
- `output_dir` (str): Output directory

**Outputs:**
- Saved files: model, scaler, encoders, metadata
- `artifact_paths` (dict): Paths to saved files

**Dependencies:** BLOCK 15 (requires final trained model)

In [ ]:
print("=" * 60)
print("MODEL PERSISTENCE - SAVING ARTIFACTS")
print("=" * 60)

# Create model directory if it doesn't exist
import os
model_dir = './models'
os.makedirs(model_dir, exist_ok=True)

# Save model
model_path = os.path.join(model_dir, 'xgboost_churn_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(best_xgb_model, f)
print(f"✓ Model saved: {model_path}")

# Save scaler
scaler_path = os.path.join(model_dir, 'scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ Scaler saved: {scaler_path}")

# Save encoders
encoders_path = os.path.join(model_dir, 'encoders.pkl')
with open(encoders_path, 'wb') as f:
    pickle.dump(encoders, f)
print(f"✓ Encoders saved: {encoders_path}")

# Save feature names
import json
features_path = os.path.join(model_dir, 'feature_names.json')
with open(features_path, 'w') as f:
    json.dump({
        'all_features': feature_names,
        'selected_features': top_features_xgb,
        'num_features': len(top_features_xgb)
    }, f, indent=2)
print(f"✓ Feature names saved: {features_path}")

# Save metadata
metadata = {
    'model_type': 'XGBoost',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'best_params': study_xgb.best_params,
    'metrics': metrics,
    'cv_scores': cv_scores.tolist(),
    'cv_mean': float(cv_scores.mean()),
    'cv_std': float(cv_scores.std()),
    'num_training_samples': len(y_resampled),
    'num_test_samples': len(y_test),
    'selected_features': top_features_xgb
}

metadata_path = os.path.join(model_dir, 'model_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Metadata saved: {metadata_path}")

print(f"\n✓ All artifacts saved successfully to '{model_dir}/'")

---
## BLOCK 18: MODEL LOADING (INFERENCE PIPELINE)
**Function:** `load_model_artifacts(model_dir)`

**Purpose:** Load saved model and preprocessing artifacts

**Inputs:**
- `model_dir` (str): Directory containing saved artifacts

**Outputs:**
- `artifacts` (dict): Dictionary with model, scaler, encoders, metadata

**Dependencies:** BLOCK 17 (requires saved artifacts)

In [ ]:
print("=" * 60)
print("MODEL LOADING - INFERENCE PIPELINE")
print("=" * 60)

def load_model_artifacts(model_dir='./models'):
    """Load all model artifacts for inference"""
    artifacts = {}
    
    # Load model
    model_path = os.path.join(model_dir, 'xgboost_churn_model.pkl')
    with open(model_path, 'rb') as f:
        artifacts['model'] = pickle.load(f)
    print(f"✓ Model loaded from: {model_path}")
    
    # Load scaler
    scaler_path = os.path.join(model_dir, 'scaler.pkl')
    with open(scaler_path, 'rb') as f:
        artifacts['scaler'] = pickle.load(f)
    print(f"✓ Scaler loaded from: {scaler_path}")
    
    # Load encoders
    encoders_path = os.path.join(model_dir, 'encoders.pkl')
    with open(encoders_path, 'rb') as f:
        artifacts['encoders'] = pickle.load(f)
    print(f"✓ Encoders loaded from: {encoders_path}")
    
    # Load feature names
    features_path = os.path.join(model_dir, 'feature_names.json')
    with open(features_path, 'r') as f:
        artifacts['feature_config'] = json.load(f)
    print(f"✓ Feature config loaded from: {features_path}")
    
    # Load metadata
    metadata_path = os.path.join(model_dir, 'model_metadata.json')
    with open(metadata_path, 'r') as f:
        artifacts['metadata'] = json.load(f)
    print(f"✓ Metadata loaded from: {metadata_path}")
    
    return artifacts

# Test loading
loaded_artifacts = load_model_artifacts('./models')

print("\nLoaded Artifacts Summary:")
print(f"  Model Type: {loaded_artifacts['metadata']['model_type']}")
print(f"  Training Date: {loaded_artifacts['metadata']['training_date']}")
print(f"  Test Accuracy: {loaded_artifacts['metadata']['metrics']['Accuracy']:.4f}")
print(f"  ROC AUC: {loaded_artifacts['metadata']['metrics']['ROC AUC']:.4f}")
print(f"  Selected Features: {loaded_artifacts['feature_config']['num_features']}")

print("\n✓ Model artifacts loaded successfully")

---
## BLOCK 19: INFERENCE FUNCTION
**Function:** `predict_churn(customer_data, artifacts)`

**Purpose:** Make predictions on new customer data

**Inputs:**
- `customer_data` (pd.DataFrame or dict): New customer data
- `artifacts` (dict): Loaded model artifacts

**Outputs:**
- `predictions` (np.ndarray): Binary predictions
- `probabilities` (np.ndarray): Churn probabilities
- `risk_category` (list): Risk level labels

**Dependencies:** BLOCK 18 (requires loaded artifacts)

In [ ]:
print("=" * 60)
print("INFERENCE FUNCTION - PREDICT CHURN")
print("=" * 60)

def predict_churn(customer_data, artifacts):
    """
    Make churn predictions on new customer data
    
    Args:
        customer_data: DataFrame or dict with customer information
        artifacts: Loaded model artifacts from load_model_artifacts()
    
    Returns:
        dict with predictions, probabilities, and risk categories
    """
    # Convert dict to DataFrame if needed
    if isinstance(customer_data, dict):
        customer_data = pd.DataFrame([customer_data])
    
    # Make a copy to avoid modifying original
    df = customer_data.copy()
    
    # 1. Encode categorical features
    # Binary encoding
    binary_mapping = {'No': 0, 'Yes': 1, False: 0, True: 1}
    if 'International plan' in df.columns:
        df['International plan'] = df['International plan'].map(binary_mapping)
    if 'Voice mail plan' in df.columns:
        df['Voice mail plan'] = df['Voice mail plan'].map(binary_mapping)
    
    # One-hot encoding for State and Area code
    if 'State' in df.columns:
        state_encoder = artifacts['encoders']['state_encoder']
        encoded_states = state_encoder.transform(df[['State']])
        state_columns = state_encoder.get_feature_names_out(['State'])
        df_states = pd.DataFrame(encoded_states, columns=state_columns, index=df.index)
        df = df.drop(['State'], axis=1)
        df = pd.concat([df, df_states], axis=1)
    
    if 'Area code' in df.columns:
        area_encoder = artifacts['encoders']['area_encoder']
        encoded_area = area_encoder.transform(df[['Area code']])
        area_columns = area_encoder.get_feature_names_out(['Area code'])
        df_area = pd.DataFrame(encoded_area, columns=area_columns, index=df.index)
        df = df.drop(['Area code'], axis=1)
        df = pd.concat([df, df_area], axis=1)
    
    # 2. Engineer features
    df['Total calls'] = (df['Total day calls'] + df['Total eve calls'] + 
                         df['Total night calls'] + df['Total intl calls'])
    df['Total charge'] = (df['Total day charge'] + df['Total eve charge'] + 
                          df['Total night charge'] + df['Total intl charge'])
    df['CScalls Rate'] = df['Customer service calls'] / df['Account length']
    
    # 3. Remove correlated features
    correlated_features = ['Total day minutes', 'Total eve minutes', 
                           'Total night minutes', 'Total intl minutes', 'Voice mail plan']
    df = df.drop(columns=[col for col in correlated_features if col in df.columns], errors='ignore')
    
    # 4. Select important features
    selected_features = artifacts['feature_config']['selected_features']
    df_selected = df[selected_features]
    
    # 5. Scale features
    scaler = artifacts['scaler']
    df_scaled = scaler.transform(df_selected)
    
    # 6. Make predictions
    model = artifacts['model']
    predictions = model.predict(df_scaled)
    probabilities = model.predict_proba(df_scaled)[:, 1]
    
    # 7. Categorize risk
    risk_categories = []
    for prob in probabilities:
        if prob < 0.3:
            risk_categories.append('Low Risk')
        elif prob < 0.7:
            risk_categories.append('Medium Risk')
        else:
            risk_categories.append('High Risk')
    
    return {
        'predictions': predictions,
        'probabilities': probabilities,
        'risk_categories': risk_categories,
        'churn_labels': ['No Churn' if p == 0 else 'Churn' for p in predictions]
    }

# Test inference with sample data from test set
sample_customer = test_df.iloc[0:1].drop('Churn', axis=1)

print("Sample Customer Data:")
print(sample_customer.head())

# Make prediction
result = predict_churn(sample_customer, loaded_artifacts)

print("\nPrediction Results:")
print(f"  Prediction: {result['churn_labels'][0]}")
print(f"  Churn Probability: {result['probabilities'][0]:.2%}")
print(f"  Risk Category: {result['risk_categories'][0]}")

print("\n✓ Inference pipeline tested successfully")

---
## SUMMARY: MODULAR PIPELINE STRUCTURE

### Identified Logical Blocks:

**1. Data Loading & Validation** (Blocks 2-3)
   - `load_data()` - Load CSV files
   - `validate_data_quality()` - Quality checks

**2. Exploratory Data Analysis** (Block 4)
   - `perform_eda()` - Generate visualizations and statistics

**3. Data Preprocessing** (Blocks 5-6)
   - `encode_categorical_features()` - Encode categorical variables
   - `remove_outliers()` - Statistical outlier removal

**4. Feature Engineering** (Blocks 7-8)
   - `engineer_features()` - Create derived features
   - `select_features_correlation()` - Remove correlated features

**5. Data Preparation** (Blocks 9-11)
   - `prepare_data_for_modeling()` - Split features/target
   - `balance_dataset()` - Handle class imbalance
   - `scale_features()` - Standardize features

**6. Model Training** (Blocks 12-13)
   - `compare_baseline_models()` - Train multiple algorithms
   - `optimize_hyperparameters()` - Tune best models

**7. Model Analysis** (Blocks 14-16)
   - `extract_feature_importance()` - Analyze feature importance
   - `evaluate_model_comprehensive()` - Full evaluation metrics
   - `cross_validate_model()` - K-fold cross-validation

**8. Model Persistence** (Blocks 17-18)
   - `save_model_artifacts()` - Save model and artifacts
   - `load_model_artifacts()` - Load for inference

**9. Inference Pipeline** (Block 19)
   - `predict_churn()` - End-to-end prediction on new data

---

**Next Steps:**
1. Convert each block into a standalone Python function
2. Create unit tests for each function
3. Build a production API (FastAPI/Flask)
4. Implement monitoring and logging
5. Set up CI/CD pipeline